# Pandas, SQL, and the Grammar of Data

In [1]:
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Populating the Database
Lets start with Relational Databases, so called because they contain "relations" (tables), which are SETS of "tuples" (rows) which map "attributes" to atomic values.

The available attributes are constrained by a "header" tuple of attributes which set the type. We do this below here.

In [2]:
ourschema="""
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
"""

## SQLite

We use sqlite here (and recommend Postgres for production purposes). Still sqlite is great for on-disk large databases which wont fit into memory.

Its also built into Python, but to use the [command line tool](https://www.sqlite.org/cli.html), I recommend you install it: https://www.sqlite.org/download.html. I also recommend you download and install the sqlite browser: http://sqlitebrowser.org .

Python implements a standard database API over all databases. Its called [DBAPI2](http://cewing.github.io/training.codefellows/lectures/day21/intro_to_dbapi2.html). It works across many SQL databases.

There is an even higher level API available, called [SQLAlchemy](http://www.sqlalchemy.org). While we wont use it here, I thoroughly recommend it, either in its direct relational form, or ORM form. Many things in Pandas use it to interface with databases. Here we'll get away with things by using SQLITE.

In [3]:
# Connect and get a DBAPI2 connection
from sqlite3 import dbapi2 as sq3
import os
PATHSTART = '.'
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db

In [4]:
# Drop tables if they exist and create them.
def init_db(dbfile, schema):
    # Creates the database tables
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    
    # Explicit commit after entire operation completed
    db.commit()
    return db

In [5]:
# Use Pandas to read in the data
dfcand = pd.read_csv("./Resources/candidates.txt", sep='|')
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


In [6]:
dfcwci = pd.read_csv("./Resources/contributors_with_candidate_id.txt", sep="|")
dfcwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [7]:
# ID is defined to auto increment, so:
del dfcwci['id']
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [8]:
# Initialise a Database
db = init_db("./Resources/cancont.db", ourschema)

### Populating w/ Pandas

In [9]:
# Append both dataframes as tables in db
dfcand.to_sql("candidates", db, if_exists="append", index=False)
dfcwci.to_sql("contributors", db, if_exists="append", index=False)

In [10]:
# Get the entire table
sel = """
SELECT * FROM candidates;
"""
c = db.cursor().execute(sel)

In [11]:
c.fetchall()

[(16, u'Mike', u'Huckabee', None, u'R'),
 (20, u'Barack', u'Obama', None, u'D'),
 (22, u'Rudolph', u'Giuliani', None, u'R'),
 (24, u'Mike', u'Gravel', None, u'D'),
 (26, u'John', u'Edwards', None, u'D'),
 (29, u'Bill', u'Richardson', None, u'D'),
 (30, u'Duncan', u'Hunter', None, u'R'),
 (31, u'Dennis', u'Kucinich', None, u'D'),
 (32, u'Ron', u'Paul', None, u'R'),
 (33, u'Joseph', u'Biden', None, u'D'),
 (34, u'Hillary', u'Clinton', u'R.', u'D'),
 (35, u'Mitt', u'Romney', None, u'R'),
 (36, u'Samuel', u'Brownback', None, u'R'),
 (37, u'John', u'McCain', None, u'R'),
 (38, u'Tom', u'Tancredo', None, u'R'),
 (39, u'Christopher', u'Dodd', u'J.', u'D'),
 (41, u'Fred', u'Thompson', u'D.', u'R')]

In [12]:
# Delete the table
rem = """
DELETE FROM candidates;
"""
c = db.cursor().execute(rem)
db.commit()
# Delete, unlike Select, actually mutates the table. So, we commit

In [13]:
c.fetchall()

[]

### Populate w/ SQL Insert 
When data > memory

In [14]:
# Create a tuple consisting of each row's data to be added to table

ins = """
INSERT INTO candidates (id, first_name, last_name, middle_name, party) \
    VALUES (?,?,?,?,?);
"""

with open("./Resources/candidates.txt") as fd:
    slines =[l.strip().split('|') for l in fd.readlines()]
    for line in slines[1:]:
        theid, first_name, last_name, middle_name, party = line
        print theid, first_name, last_name, middle_name, party
        valstoinsert = (int(theid), first_name, last_name, middle_name, party)
        print ins, valstoinsert
        db.cursor().execute(ins, valstoinsert)

33 Joseph Biden  D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
(33, 'Joseph', 'Biden', '', 'D')
36 Samuel Brownback  R

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
(36, 'Samuel', 'Brownback', '', 'R')
34 Hillary Clinton R. D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
(34, 'Hillary', 'Clinton', 'R.', 'D')
39 Christopher Dodd J. D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
(39, 'Christopher', 'Dodd', 'J.', 'D')
26 John Edwards  D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
(26, 'John', 'Edwards', '', 'D')
22 Rudolph Giuliani  R

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
(22, 'Rudolph', 'Giuliani', '', 'R')
24 Mike Gravel  D

INSERT INTO candidates (id, first_name, last_name, midd

In [15]:
def make_query(sel):
    c = db.cursor().execute(sel)
    return c.fetchall()

In [16]:
make_query("SELECT * FROM candidates;")

[(16, u'Mike', u'Huckabee', u'', u'R'),
 (20, u'Barack', u'Obama', u'', u'D'),
 (22, u'Rudolph', u'Giuliani', u'', u'R'),
 (24, u'Mike', u'Gravel', u'', u'D'),
 (26, u'John', u'Edwards', u'', u'D'),
 (29, u'Bill', u'Richardson', u'', u'D'),
 (30, u'Duncan', u'Hunter', u'', u'R'),
 (31, u'Dennis', u'Kucinich', u'', u'D'),
 (32, u'Ron', u'Paul', u'', u'R'),
 (33, u'Joseph', u'Biden', u'', u'D'),
 (34, u'Hillary', u'Clinton', u'R.', u'D'),
 (35, u'Mitt', u'Romney', u'', u'R'),
 (36, u'Samuel', u'Brownback', u'', u'R'),
 (37, u'John', u'McCain', u'', u'R'),
 (38, u'Tom', u'Tancredo', u'', u'R'),
 (39, u'Christopher', u'Dodd', u'J.', u'D'),
 (41, u'Fred', u'Thompson', u'D.', u'R')]

## Single Table Verbs
Let us now focus on core data manipulation commands. The reason to do this is that they are ***universal across systems, and by identifying them, we can quickly ask how to do these*** when we encounter a new system.

See https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/ which has a comparison of r/dplyr and pandas. I stole and modified this table from there:

``dplyr`` has a small set of nicely defined verbs. I've listed their closest pandas verbs.


<table>
  <tr>
    <th><b>VERB</b></th>
    <th><b>dplyr</b></th>
    <th><b>pandas</b></th>
    <th><b>SQL</b></th>
  </tr>
  <tr>
    <td>QUERY/SELECTION</td>
    <td>filter() (and slice())</td>
    <td>query() (and loc[], iloc[])</td>
    <td>SELECT WHERE</td>
  </tr>
  <tr>
    <td>SORT</td>
    <td>arrange()</td>
    <td>sort()</td>
    <td>ORDER BY</td>
  </tr>
  <tr>
    <td>SELECT-COLUMNS/PROJECTION</td>
    <td>select() (and rename())</td>
    <td>[](__getitem__) (and rename())</td>
    <td>SELECT COLUMN</td>
  </tr>
  <tr>
    <td>SELECT-DISTINCT</td>
    <td>distinct()</td>
    <td>unique(),drop_duplicates()</td>
    <td>SELECT DISTINCT COLUMN</td>
  </tr>
  <tr>
    <td>ASSIGN</td>
    <td>mutate() (and transmute())</td>
    <td>assign</td>
    <td>ALTER/UPDATE</td>
  </tr>
  <tr>
    <td>AGGREGATE</td>
    <td>summarise()</td>
    <td>describe(), mean(), max()</td>
    <td>None, AVG(),MAX()</td>
  </tr>
  <tr>
    <td>SAMPLE</td>
    <td>sample_n() and sample_frac()</td>
    <td>sample()</td>
    <td>implementation dep, use RAND()</td>
  </tr>
  <tr>
    <td>GROUP-AGG</td>
    <td>group_by/summarize</td>
    <td>groupby/agg, count, mean</td>
    <td>GROUP BY</td>
  </tr>
  <tr>
    <td>DELETE</td>
    <td>?</td>
    <td>drop/masking</td>
    <td>DELETE/WHERE</td>
  </tr>
</table>